In [1]:
import pandas as pd
import numpy as np

In [2]:
def fix_str_float(ds, col):
    ds[col] = ds[col].str.replace(r'[^0-9\.]','') #reemplazo todo por espacio 
    ds[col] = np.where(ds[col]=='',np.nan,ds[col]) #el espacio vacio lo convierto al valor de la columna
    ds[col] = ds[col].astype('float32') #lo convierto en float
    return ds[col].astype('float32')

In [ ]:
clicks_df = pd.read_csv('data/clicks.csv',dtype={'advertiser_id':'int32','action_id':'float32','source_id':'int32','country_code':'category',\
                                                 'latitude':'float32','longitude':'float32','carrier_id':'float32','os_minor':'category',\
                                                  'os_major':'category','specs_brand':'category','timeToClick':'float32','ref_type':'category'})

clicks_df['touchX'] = fix_str_float(clicks_df,'touchX')
clicks_df['touchY'] = fix_str_float(clicks_df,'touchY')
clicks_df['created'] = pd.to_datetime(clicks_df['created'])
events_df = pd.read_csv('data/events.csv', low_memory=False, dtype={'event_id':'int32','ref_type':'category','application_id':'int32',\
                                                                                            'attributed':'bool','device_countrycode':'category','device_city':'category',\
                                                                                            'trans_id':'category','carrier':'category','device_os':'category',\
                                                                                            'connection_type':'category'})
events_df['date'] = pd.to_datetime(events_df['date'])
events_df['wifi'].astype('bool', inplace=True)
installs_df = pd.read_csv('data/installs.csv', dtype={'ref_type':'category','application_id':'int32',\
                                                      'device_brand':'category'})
installs_df.drop(columns=['session_user_agent'], inplace=True)
installs_df['wifi'].astype('bool', inplace=True)
installs_df['created'] = pd.to_datetime(installs_df['created'])
auctions_df = pd.read_csv('data/auctions.csv', low_memory=False, dtype={'country':'category','platform':'category','ref_type_id':'category'})

auctions_df['date'] = pd.to_datetime(auctions_df['date'])
print('setup done')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
events_df.head()

,index,date,event_id,ref_type,ref_hash,application_id,attributed,device_countrycode,device_os_version,device_brand,...,trans_id,user_agent,event_uuid,carrier,kind,device_os,wifi,connection_type,ip_address,device_language
0,2130678,2019-04-20 01:42:49.120,0,1891515180541284343,5857744372586891366,210,False,6287817205707153877,NaN,NaN,...,NaN,5.046185e+18,5b506964-5f47-4b28-a8c2-8a92d6c23379,NaN,5.882882e+18,NaN,False,NaN,7544543351571901618,3.301378e+18
1,2130680,2019-04-20 01:42:49.340,1,1891515180541284343,7642521036780133571,210,False,6287817205707153877,NaN,NaN,...,NaN,NaN,f1fb9d15-1a7b-4116-8d3b-c4c403e197e2,NaN,4.017674e+18,NaN,False,NaN,6949523255335024165,NaN
2,2130681,2019-04-20 01:42:49.365,1,1891515180541284343,2548841562898283198,210,False,6287817205707153877,NaN,NaN,...,NaN,NaN,c85a0b15-a5d7-472e-8116-6bfa3db19687,NaN,4.017674e+18,NaN,False,NaN,6428537280982666957,NaN
3,2130684,2019-04-20 01:42:51.438,2,1891515180541284343,609402887625919085,210,False,6287817205707153877,NaN,NaN,...,NaN,NaN,f4aa0a97-2de6-4f22-95c6-1b3150112cb9,NaN,6.168309e+18,NaN,False,NaN,7607371352198017145,NaN
4,2130688,2019-04-20 01:42:51.838,1,1891515180541284343,9114651763556439823,210,False,6287817205707153877,NaN,NaN,...,NaN,NaN,08e2f7f7-875f-4aa0-b337-b9b87b0d83ea,NaN,4.017674e+18,NaN,False,NaN,2901772839007473756,NaN
